<a href="https://colab.research.google.com/github/KrungSalad/Google_Colab/blob/master/Lab_ML_3_JN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! pip install keras

In [0]:
%tensorflow_version 1.4

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.4`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
import tensorflow
print(tensorflow.__version__)

1.15.2


In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

from sklearn import model_selection
from sklearn.model_selection import train_test_split,\
    StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, \
    confusion_matrix

from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LSTM, Flatten, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow

# for heatmap purpose
import seaborn as sns
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
pd.options.display.width = None
pd.options.display.max_rows = None

In [0]:
Chengdu = pd.read_csv('ChengduPM20100101_20151231.csv')
dfChengdu = pd.DataFrame(Chengdu)
Shenyang = pd.read_csv('ShenyangPM20100101_20151231.csv')
dfShenyang = pd.DataFrame(Shenyang)

In [0]:
t1 = pd.to_datetime(dfChengdu[['year','month','day','hour']])
t2 = pd.to_datetime(dfShenyang[['year','month','day','hour']])

dfChengdu = dfChengdu[['PM_US Post','HUMI','Iws','Iprec']]
dfShenyang = dfShenyang[['PM_US Post','HUMI','Iws','Iprec']]
dfChengdu['Time'] = t1
dfShenyang['Time'] = t2

dfChengdu = dfChengdu.fillna(dfChengdu.mean())
dfChengdu =dfChengdu.interpolate(method='polynomial', order=2)
dfShenyang = dfShenyang.fillna(dfShenyang.mean())
dfShenyang = dfShenyang.interpolate(method='polynomial', order=2)

print(dfChengdu)
Chengdu = dfChengdu[['PM_US Post','HUMI','Iws','Iprec']].to_numpy()
chx, chy = Chengdu.shape 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
#choose  time step=3 stride=2

timeSeries = []
stack = []
for i in range(chx):
    ele = []
    l = []
    for j in range(chy):
        l.append(Chengdu[i][j])
    stack.append(l)
stack = np.array(stack)
print(stack, stack.shape)
k = 0
while k<chx-6 :
    t1 = []
    for j in range(3):
        if k+j < chx:
            t1.append(stack[k+j])
    timeSeries.append(t1)
    k += 2


timeSeries = np.array(timeSeries)
print(timeSeries, timeSeries.shape)

[[ 83.40761246  81.2          1.           0.        ]
 [ 83.40761246  86.99         1.           0.        ]
 [ 83.40761246  86.99         1.           0.        ]
 ...
 [213.          87.28         1.           0.        ]
 [236.          93.4          2.           0.        ]
 [249.          93.4          4.           0.        ]] (52584, 4)
[[[ 83.40761246  81.2          1.           0.        ]
  [ 83.40761246  86.99         1.           0.        ]
  [ 83.40761246  86.99         1.           0.        ]]

 [[ 83.40761246  86.99         1.           0.        ]
  [ 83.40761246  86.89         1.           0.        ]
  [ 83.40761246  86.79         1.           0.        ]]

 [[ 83.40761246  86.79         1.           0.        ]
  [ 83.40761246  93.13         1.           0.        ]
  [ 83.40761246  93.13         1.           0.        ]]

 ...

 [[230.          87.28         1.           0.        ]
  [280.          81.61         1.           0.        ]
  [302.          81.61   

In [0]:
# prepare groud truth by majority vote
# and one-hot encoding (for y)

def find_majority(votes):
    vote_count = Counter(votes)
    top_two = vote_count.most_common(2)
    if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        # It is a tie
        return 0
    return top_two[0][0]
#print(timeSeries[1][3][0])
dx, dy, dz = timeSeries.shape
y = []
for i in range(dx):
    l = []
    for j in range(dy):
        if timeSeries[i][j][0] < 10:
            l.append(0)
        elif timeSeries[i][j][0] < 25:
            l.append(1)
        else:
            l.append(2)
    mjr = find_majority(l)
    x = [0, 0, 0]
    x[mjr] = 1
    y.append(x)

X = timeSeries
y = np.array(y)
print(y)

[[0 0 1]
 [0 0 1]
 [0 0 1]
 ...
 [0 0 1]
 [0 0 1]
 [0 0 1]]


In [0]:
X_train, X_val, y_train, y_val = train_test_split(\
    X, y, test_size=0.3, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(\
    X_val, y_val, test_size=0.3, random_state=42)

X_train2 = X_train
y_train2 = y_train
X_val2 = X_val
y_val2 = y_val
X_test2 = X_test
y_test2 = y_test

# Section 2


In [0]:

def model1func(optimizer='adam'):
    model = Sequential()

    model.add(Conv2D(5, kernel_size=(3,3),\
        activation='relu', input_shape=(3, 4, 1),\
        padding='same'))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(5, activation= 'relu'))
    model.add(Dense(3, activation='sigmoid'))
    model.summary()

    model.compile( loss='categorical_crossentropy',\
    optimizer='Adam', metrics=['accuracy'])

    return model

In [0]:
# have to change to 4 dimensions
a, b, c = X_train.shape
X_train = X_train.reshape([a, b, c, 1])

a, b, c = X_val.shape
X_val = X_val.reshape([a, b, c, 1])

model = model1func()
history = model.fit(X_train, y_train,\
    batch_size=3,\
    validation_data=(X_val, y_val),\
    epochs=2)

a, b, c = X_test.shape
X_test = X_test.reshape([a, b, c, 1])
y_prediction = model. predict(X_test)

print(y_prediction)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 3, 4, 5)           50        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 1, 5)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1, 5)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 18        


In [0]:

model2 = Sequential()

model2.add(LSTM(32, input_shape=(3, 4)))
# Optional: model.add(BatchNormalization())
model2.add(Dropout(0.2))

model2.add(Dense(3, activation='sigmoid'))
model2.summary()

model2.compile( loss='categorical_crossentropy',
 optimizer='Adam', metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                4736      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 4,835
Trainable params: 4,835
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Training the model
history2 = model2.fit( X_train2, y_train2,
 validation_data=(X_val2, y_val2),
 epochs=1)

y_prediction2 = model2. predict(X_test2)
print(y_prediction2)


Train on 18402 samples, validate on 5520 samples
Epoch 1/1
18402/18402 [==============================] - 3s 172us/step - loss: 0.1558 - acc: 0.9727 - val_loss: 0.0545 - val_acc: 0.9817
[[0.00274855 0.03243083 0.853155  ]
 [0.00757658 0.07545894 0.7113829 ]
 [0.00333911 0.00499663 0.9071179 ]
 ...
 [0.00333962 0.00499701 0.90711206]
 [0.00334141 0.00499844 0.90708786]
 [0.00333943 0.00499688 0.90711504]]


In [0]:
import folium
from folium import plugins

In [0]:
# convert form sigmoid result to [0,0,1]
y_pred_single = [np.argmax(p) for p in y_prediction]
y_test_single=[np.argmax(p) for p in y_test]
y_pred_single2 = [np.argmax(p) for p in y_prediction2]
y_test_single2=[np.argmax(p) for p in y_test2]

r1 = classification_report(y_test_single, y_pred_single)
print('Report for CNN is!!\n', r1)
r2 = classification_report(y_test_single2, y_pred_single2)
print('Report for LSTM is!!\n', r2)
conf1 = confusion_matrix(y_test_single, y_pred_single)
print('Confusion Matrix for CNN is!!\n', conf1)
conf2 = confusion_matrix(y_test_single2, y_pred_single2)
print('Confusion Matrix for LSTM is!!\n', conf2)

Report for CNN is!!
               precision    recall  f1-score   support

           0       0.00      1.00      0.01         7
           1       0.00      0.00      0.00        66
           2       0.00      0.00      0.00      2294

    accuracy                           0.00      2367
   macro avg       0.00      0.33      0.00      2367
weighted avg       0.00      0.00      0.00      2367

Report for LSTM is!!
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         7
           1       0.82      0.55      0.65        66
           2       0.99      1.00      0.99      2294

    accuracy                           0.98      2367
   macro avg       0.60      0.51      0.55      2367
weighted avg       0.98      0.98      0.98      2367

Confusion Matrix for CNN is!!
 [[   7    0    0]
 [  66    0    0]
 [2294    0    0]]
Confusion Matrix for LSTM is!!
 [[   0    5    2]
 [   0   36   30]
 [   0    3 2291]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:

model = KerasClassifier(build_fn=model1func)

k = 5
seed = 11
skfold = model_selection.StratifiedKFold(n_splits=k, shuffle = True,
random_state=seed)
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
optimizers = ['adam','adadelta']

param_grid = dict(batch_size=batch_size, \
    epochs=epochs, optimizer=optimizers)

gsCV_model = model_selection.GridSearchCV(estimator=model,\
param_grid=param_grid, n_jobs=2, cv=skfold, scoring='accuracy')

a, b = y_train.shape
y_train_1D = []
for i in range(a):
    for j in range(b):
        if y_train[i][j] == 1:
            y_train_1D.append(j)

y_train = np.array(y_train_1D)
print(y_train, y_train.shape)
gsCV_model.fit(X_train, y_train)

gridsearch_result = pd.DataFrame(gsCV_model.cv_results_)
print(gridsearch_result)
gridsearch_result.to_csv('FileLab3.csv')

print(gsCV_model.best_params_)
print(gsCV_model.best_score_)

a, b = y_test.shape
y_test_1D = []
for i in range(a):
    for j in range(b):
        if y_test[i][j] == 1:
            y_test_1D.append(j)

y_prediction_best_param = gsCV_model.predict(X_test)

print('before', y_prediction)
print('After', y_prediction_best_param)

r1_b = classification_report(y_test_1D, y_prediction_best_param)
conf1_b = confusion_matrix(y_test_1D, y_prediction_best_param)
print('before', r1)
print(conf1)
print('After', r1_b)
print(conf1_b)

[2 2 2 ... 2 2 2] (18402,)


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
